# Inverse Optimization for DFS
_Applying Ghobadi and Mahmoudzadeh 2021_

In [ ]:
using JuMP
using Gurobi
using LinearAlgebra
using Distributions
using Random

Random.seed!(42)

In [ ]:
using Revise

includet("models/forward.jl")
import .Forward as Forward

includet("models/inversedemand.jl")
import .InverseDemand as IODemand

includet("models/inverselinreg.jl")
import .InverseLinReg as IOLinReg

includet("datagen/data-generation.jl")
import .DataGeneration as DataGen

In [ ]:
gurobi_env = Gurobi.Env()

## Forward problem

In [ ]:
function forward_example_params()::Forward.Params
    enabled_flows = ones(Bool, (2, 2))
    enabled_flows[1, 2] = false

    return Forward.Params(
        n_paths=2, 
        n_commodities=2,
        capacities=[1000, 15],
        design_costs=[10000, 100],
        flow_costs=[100 100 ; 10 10],
        enabled_flows=enabled_flows
    )
end

function forward_example_demand()::Vector
    return [10, 6]
end

In [ ]:
forward_params = forward_example_params()
forward_demand = forward_example_demand()

forward_sol = Forward.create_and_solve_problem(forward_params, forward_demand, gurobi_env=gurobi_env)

In [ ]:
println(forward_sol.x_sol)
println(forward_sol.z_sol)

## Inverse problem

### Demand only

In [ ]:
inverse_demand_model = IODemand.create_problem(forward_params, forward_sol)
latex_formulation(inverse_demand_model)

In [ ]:
inverse_demand_solution = IODemand.solve_problem!(inverse_demand_model)

In [ ]:
println("Found demands:")
show(inverse_demand_solution.demands)
println("\nActual demands:")
show(forward_demand)

### Linear regression without noise

In [ ]:
function inverse_linreg_example_datagen_params()
    return DataGen.DataGenParams(weights=[1.5 2 1; 1 1 0.4])
end

function inverse_linreg_example_problem_params()
    return IOLinReg.Params(n_features=3, forward_params=forward_example_params())
end

function inverse_linreg_example_featuress()
    return [4 2 1; 3 1 5; 0.2 4 1]'
end

In [ ]:
linreg_datagen_params = inverse_linreg_example_datagen_params()
linreg_problem_params = inverse_linreg_example_problem_params()
linreg_input_featuress = inverse_linreg_example_featuress()

linreg_noises = DataGen.generate_noises(linreg_datagen_params, size(linreg_input_featuress)[2])
linreg_demands = DataGen.generate_demands(linreg_datagen_params, linreg_input_featuress, linreg_noises)
linreg_solution_points = DataGen.generate_solution_points(linreg_input_featuress, linreg_demands, linreg_problem_params)

In [ ]:
linreg_model = IOLinReg.create_problem(linreg_problem_params, linreg_solution_points)
latex_formulation(linreg_model)

In [ ]:
linreg_solution = IOLinReg.solve_problem!(linreg_model, linreg_problem_params)

In [ ]:
println("Found weights:")
show(linreg_solution.weights)
println("\nActual weights:")
show(linreg_datagen_params.weights)

### Linear regression with noise 

In [ ]:
function noisy_example_datagen_params()
    return DataGen.DataGenParams(weights=[1.5 2 1; 1 1 0.4], noise_variance=[2.5, 1.2])
end

function noisy_example_problem_params()
    return IOLinReg.Params(n_features=3, forward_params=forward_example_params(), with_noise=true)
end

function noisy_example_featuress(datagen_params::DataGen.DataGenParams, n_points::Integer)
    return DataGen.generate_input_features(datagen_params, n_points, upper_bound=5)
end

In [ ]:
n_points = 30

noisy_datagen_params = noisy_example_datagen_params()
noisy_problem_params = noisy_example_problem_params()
noisy_input_featuress = noisy_example_featuress(noisy_datagen_params, n_points)

noisy_noises = DataGen.generate_noises(noisy_datagen_params, n_points)
noisy_demands = DataGen.generate_demands(noisy_datagen_params, noisy_input_featuress, noisy_noises)
noisy_solution_points = DataGen.generate_solution_points(noisy_input_featuress, noisy_demands, noisy_problem_params, gurobi_env=gurobi_env)

In [ ]:
using Plots
using LaTeXStrings

function anim_plot_demand(solutions, noisy_datagen_params, demand_index=1)
    phis = [solution.linreg_features for solution in solutions]

    phi1 = getindex.(phis, 1)
    phi2 = getindex.(phis, 2)
    demands = [sol.actual_demands[demand_index] for sol in solutions]

    xs = [0, 5]
    ys = [0, 5]
    d(p1, p2) = noisy_datagen_params.weights[demand_index, :]' * [p1, p2]

    plt = surface(xs, ys, d.(xs', ys), xlabel=L"\phi_1", ylabel=L"\phi_2", zlabel=L"d_%$(demand_index)")
    scatter3d!(phi1, phi2, demands, labels="Actual demands")

    anim = @animate for i in vcat(30:100, 100:-1:30)
        plot!(plt, camera = (i, 10))
    end

    return gif(anim, "img/animsurf.gif", fps = 15)
end

# Only works when n_features = 2
# anim_plot_demand(noisy_solution_points, noisy_datagen_params)

In [ ]:
noisy_model = IOLinReg.create_problem(noisy_problem_params, noisy_solution_points)

In [ ]:
noisy_solution = IOLinReg.solve_problem!(noisy_model, noisy_problem_params)

In [ ]:
println("Found weights:")
show(noisy_solution.weights)
println("\nActual weights:")
show(noisy_datagen_params.weights)

println("\n\nRMSE:")
show(noisy_solution.rmse)

## Experiments

In [ ]:
function predict_example_weights(n_points)
    datagen_params = noisy_example_datagen_params()
    problem_params = noisy_example_problem_params()
    input_featuress = noisy_example_featuress(datagen_params, n_points)
    
    noises = DataGen.generate_noises(datagen_params, n_points)
    demands = DataGen.generate_demands(datagen_params, input_featuress, noises)
    solution_points = DataGen.generate_solution_points(input_featuress, demands, problem_params, gurobi_env=gurobi_env)

    noisy_model = IOLinReg.create_problem(problem_params, solution_points; gurobi_env=gurobi_env)
    set_silent(noisy_model)

    return IOLinReg.solve_problem!(noisy_model, problem_params)
end

In [ ]:
max_n_datapoints= 200
n_datapoints = vcat([2, 3, 4], 5:5:max_n_datapoints)
solutions = [predict_example_weights(n) for n in n_datapoints]

In [ ]:
avg_noise_variance = sqrt(mean(noisy_datagen_params.noise_variance))

plt = plot(n_datapoints, [sol.rmse for sol in solutions], xlim=(2, max_n_datapoints), xlabel="Number of datapoints", ylabel="RMSE", label="IO Model")
plot!([2, max_n_datapoints], [avg_noise_variance, avg_noise_variance], linestyle=:dash, label="Demand average std dev")

savefig(plt, "img/io-mse.png")
plt

In [ ]:
frobenius_norms = [norm(sol.weights .- noisy_datagen_params.weights) for sol in solutions]
plt2 = plot(n_datapoints, frobenius_norms, xlim=(2, max_n_datapoints), ylim=(0, 3.5), xlabel="Number of datapoints", ylabel=L"\Vert \Omega - \hat{\Omega} \Vert_F", label="IO Model")

savefig(plt2, "img/frobenius.png")
plt2